In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!nvidia-smi

Thu Jul 27 07:56:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd /kaggle/working

/kaggle/working


In [4]:
# !pip install torch==1.7.0 torchvision==0.8.0
!pip install torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 811.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 79.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1
    Uninstalling torchaudio-2.0.1:
      Successfully uninstalled torchaudio-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.


In [5]:
!pip install -U openmim
# Install mmengine
!mim install mmengine
# Install MMCV
!mim install 'mmcv >= 2.0.0rc1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.1 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 11.7 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 4.8 MB/s eta 0:00:00


In [6]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)

1.12.0+cu102 True
0.13.0+cu102


In [7]:
!rm -r mmsegmentation
!git clone https://github.com/alirafiqmalik/mmsegmentation.git 
%cd mmsegmentation
# !pip install -v -e .
!python setup.py build
!pip install .
%cd ..

rm: cannot remove 'mmsegmentation': No such file or directory
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 1530, done.
remote: Counting objects: 100% (1530/1530), done.
remote: Compressing objects: 100% (765/765), done.
remote: Total 1530 (delta 771), reused 1502 (delta 743), pack-reused 0
Receiving objects: 100% (1530/1530), 5.36 MiB | 21.29 MiB/s, done.
Resolving deltas: 100% (771/771), done.
/kaggle/working/mmsegmentation
running build
running build_py
creating build
creating build/lib
creating build/lib/mmseg
copying mmseg/version.py -> build/lib/mmseg
copying mmseg/__init__.py -> build/lib/mmseg
creating build/lib/tests
copying tests/test_config.py -> build/lib/tests
copying tests/test_sampler.py -> build/lib/tests
copying tests/__init__.py -> build/lib/tests
copying tests/test_digit_version.py -> build/lib/tests
creating build/lib/mmseg/structures
copying mmseg/structures/__init__.py -> build/lib/mmseg/structures
copying mmseg/structures/seg_data_sample.py -> bui

In [8]:
# Check MMSegmentation installation
import mmseg,mmcv,mmengine
print(mmseg.__version__,mmcv.__version__,mmengine.__version__)

1.0.0 2.0.1 0.8.2


In [9]:
# Let's take a look at the dataset
import mmcv
import mmengine
import matplotlib.pyplot as plt
import os

In [10]:
# define dataset root and directory for images and annotations
data_root = '/kaggle/input/floodnet-mydata-output12gb/content/drive/MyDrive/Thesis_Code_msee/FloodNet_myData'
img_dir = 'images'
ann_dir = 'annotations'

In [11]:
import pandas as pd
# define class and palette for better visualization
df=pd.read_csv('/kaggle/input/floodnet-mydata-output12gb/content/drive/MyDrive/Thesis_Code_msee/FloodNet_myData/Now_class_dict_seg_10clss.csv')
classes = df['name']
palette = df[[' r', ' g', ' b']].values
id2label = classes.to_dict()
label2id = {v: k for k, v in id2label.items()}
print("classes", len(id2label))
print(classes)
print(palette)
classes=list(classes)
palette=list(palette)
num_classes=len(id2label)

classes 10
0         Background
1    BuildingFlooded
2        BNonFlooded
3        RoadFlooded
4        RNonFlooded
5              Water
6               Tree
7             Vecile
8               Pool
9              Grass
Name: name, dtype: object
[[  0   0   0]
 [255   0   0]
 [181  72  72]
 [150 150   0]
 [135 135 135]
 [  0 224 224]
 [  0   0 225]
 [204   0 204]
 [237 237   0]
 [  0 225   0]]


In [12]:
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset


@DATASETS.register_module()
class ImageSegmentationDataset(BaseSegDataset):
    METAINFO = dict(classes = classes, palette = palette)
    def __init__(self, **kwargs):
        super().__init__(img_suffix='.jpg', seg_map_suffix="_lab.png", **kwargs)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
from mmengine import Config
cfg = Config.fromfile('/kaggle/input/trained-models-swint-finetuning-folk/checkpoint/swin-base-patch4-window12-in1k-384x384-pre_upernet_8xb2-160k_ade20k-512x512.py')
# cfg.checkpoint_file="/home/haris/Desktop/checkpoint/iter_9000.pth"
# cfg.model.backbone.init_cfg=cfg.checkpoint_file
cfg.load_from = "/kaggle/input/trained-models-swint-finetuning-folk/checkpoint/iter_6000.pth"

In [14]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

07/27 07:58:44 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 1.12.0+cu102
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;ar

/opt/conda/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/opt/conda/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


07/27 07:58:53 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
07/27 07:58:53 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/opt/conda/lib/python3.10/site-packages/mmseg/engine/hooks/visualization_hook.py:61: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '


In [15]:
runner.test()

/opt/conda/lib/python3.10/site-packages/mmseg/datasets/transforms/loading.py:78: UserWarning: `reduce_zero_label` will be deprecated, if you would like to ignore the zero label, please set `reduce_zero_label=True` when dataset initialized
  warnings.warn('`reduce_zero_label` will be deprecated, '


07/27 07:58:56 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.
Loads checkpoint by local backend from path: /kaggle/input/trained-models-swint-finetuning-folk/checkpoint/iter_6000.pth
07/27 07:59:17 - mmengine - INFO - Load checkpoint from /kaggle/input/trained-models-swint-finetuning-folk/checkpoint/iter_6000.pth
07/27 08:00:09 - mmengine - INFO - Iter(test) [100/450]    eta: 0:03:04  time: 0.4885  data_time: 0.2516  memory: 9209  
07/27 08:01:04 - mmengine - INFO - Iter(test) [200/450]    eta: 0:02:13  time: 0.6142  data_time: 0.3707  memory: 9214  
07/27 08:01:57 - mmengine - INFO - Iter(test) [300/450]    eta: 0:01:19  time: 0.4998  data_time: 0.2422  memory: 9203  
07/27 08:02:47 - mmengine - INFO - Iter(test) [400/450]    eta: 0:00:26  time: 0.4948  data_time: 0.2418  memory: 7910  
07/27 08:03:12 - mmengine - INFO - per class results:
07/27 08:03:12 - mmengine - INFO - 
+-----------------+-------+-------+-------+--------+-----------+--------+
|      Class

{'aAcc': 87.11,
 'mIoU': 61.06,
 'mAcc': 73.75,
 'mDice': 73.31,
 'mFscore': 73.31,
 'mPrecision': 75.2,
 'mRecall': 73.75}